In [16]:
# %% [markdown]
# ## Leitura dos arquivos de log (com skip e limite de linhas)

# %%
# Caminhos dos arquivos (ajuste se necessário)
pairs_path = "/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs.txt"
bins_path  = "/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs_bin.txt"

# %%
def print_lines(filepath, n=10, skip=0):
    """
    Imprime n linhas de um arquivo de texto, pulando as primeiras `skip`.
    """
    print(f"\n=== {filepath} (pulando {skip} linhas, imprimindo {n}) ===")
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            # Pula as linhas iniciais
            for _ in range(skip):
                next(f, None)
            # Lê e imprime as próximas n linhas
            for i in range(n):
                line = f.readline()
                if not line:
                    break
                print(line.strip())
    except FileNotFoundError:
        print("Arquivo não encontrado.")

# %%
# Exemplo: imprimir 10 linhas a partir da linha 11 (pulando as 10 primeiras)
print_lines(pairs_path, n=100, skip=1500000)
print_lines(bins_path, n=100, skip=1500000)



=== /home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs.txt (pulando 1500000 linhas, imprimindo 100) ===
0.31640625 0.2467041015625
0.06671142578125 0.03778076171875
0.1055908203125 0.06378173828125
0.141357421875 0.09014892578125
0.051849365234375 0.0286102294921875
0.0194091796875 0.0100860595703125
0.160400390625 0.1051025390625
0.279052734375 0.210205078125
0.0718994140625 0.04107666015625
0.016937255859375 0.008758544921875
0.0697021484375 0.0396728515625
0.233154296875 0.1673583984375
0.06915283203125 0.039337158203125
-0.004150390625 -0.0020580291748046875
0.1944580078125 0.1331787109375
0.2080078125 0.1448974609375
0.099609375 0.0595703125
0.07861328125 0.045440673828125
0.1907958984375 0.130126953125
0.283935546875 0.21484375
0.279296875 0.21044921875
0.20263671875 0.1402587890625
0.50732421875 0.448486328125
0.2646484375 0.1964111328125
0.398193359375 0.330810546875
0.580078125 0.5283203125
0.61865234375 0.57080078125
0.619140625 0.5712890625
0.473388671875 0.4113769

In [17]:
# %% [markdown]
# ## Contagem de linhas com avisos a cada 100.000

# %%
# Caminho do arquivo (ajuste se necessário)
filepath = "/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs.txt"

# %%
def count_lines_with_progress(filepath, step=100_000):
    """
    Conta o número de linhas em um arquivo de texto.
    Exibe um aviso a cada `step` linhas processadas.
    """
    total = 0
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            for total, _ in enumerate(f, start=1):
                if total % step == 0:
                    print(f"[Progresso] Já percorreu {total:,} linhas...")
        print(f"\n[Finalizado] Número total de linhas: {total:,}")
    except FileNotFoundError:
        print("Arquivo não encontrado.")
    return total

# %%
# Executa a contagem
total_linhas = count_lines_with_progress(filepath, step=100_000)


[Progresso] Já percorreu 100,000 linhas...
[Progresso] Já percorreu 200,000 linhas...
[Progresso] Já percorreu 300,000 linhas...
[Progresso] Já percorreu 400,000 linhas...
[Progresso] Já percorreu 500,000 linhas...
[Progresso] Já percorreu 600,000 linhas...
[Progresso] Já percorreu 700,000 linhas...
[Progresso] Já percorreu 800,000 linhas...
[Progresso] Já percorreu 900,000 linhas...
[Progresso] Já percorreu 1,000,000 linhas...
[Progresso] Já percorreu 1,100,000 linhas...
[Progresso] Já percorreu 1,200,000 linhas...
[Progresso] Já percorreu 1,300,000 linhas...
[Progresso] Já percorreu 1,400,000 linhas...
[Progresso] Já percorreu 1,500,000 linhas...
[Progresso] Já percorreu 1,600,000 linhas...
[Progresso] Já percorreu 1,700,000 linhas...
[Progresso] Já percorreu 1,800,000 linhas...
[Progresso] Já percorreu 1,900,000 linhas...
[Progresso] Já percorreu 2,000,000 linhas...
[Progresso] Já percorreu 2,100,000 linhas...
[Progresso] Já percorreu 2,200,000 linhas...
[Progresso] Já percorreu 2,3

In [8]:
import json
import random
from typing import Iterable, Any, List, Sequence, Optional
import numpy as np

# --------- utilitários ---------

def iter_scalars(nested: Any) -> Iterable[float]:
    """Percorre recursivamente listas aninhadas e gera escalares numéricos."""
    if isinstance(nested, list):
        for item in nested:
            yield from iter_scalars(item)
    else:
        try:
            yield float(nested)
        except (TypeError, ValueError):
            return

# --------- 1) carrega todas as linhas para RAM ---------

def load_jsonl_lines(path: str) -> List[str]:
    """
    Lê o JSONL inteiro e retorna uma lista de linhas não vazias.
    Útil se quiser fazer parsing depois, várias vezes, sem reler do disco.
    """
    with open(path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    return lines

# --------- 2) parseia tudo para objetos Python ---------

def parse_jsonl_objects(lines: Sequence[str]) -> List[dict]:
    """
    Recebe as linhas (já em RAM) e parseia cada uma como JSON.
    Retorna lista de dicts. Linhas inválidas são ignoradas.
    """
    objs = []
    for i, ln in enumerate(lines, 1):
        try:
            objs.append(json.loads(ln))
        except json.JSONDecodeError:
            # Se quiser, faça log: print(f"[WARN] Linha {i}: JSON inválido — ignorando.")
            pass
    return objs

# --------- 3) extrai todos os escalares de 'data' e devolve um array ---------

def extract_all_scalars_from_objects(objs: Sequence[dict],
                                     field: str = "data",
                                     dtype: str = "float32") -> np.ndarray:
    """
    Percorre todos os objetos, coleta e achata o campo 'data' (listas aninhadas),
    devolvendo um array NumPy (por padrão float32 para ser mais leve na RAM).
    """
    vals: List[float] = []
    for obj in objs:
        data = obj.get(field, None)
        if data is None:
            continue
        for v in iter_scalars(data):
            vals.append(v)
    if not vals:
        return np.array([], dtype=dtype)
    return np.array(vals, dtype=dtype)

# --------- amostragem/exportação operando 100% em RAM ---------

def sample_values_in_memory(values: np.ndarray,
                            num_values: int,
                            random_sample: bool = False,
                            seed: Optional[int] = None) -> np.ndarray:
    """
    Seleciona valores de um array em RAM: primeiros N ou aleatórios sem reposição.
    """
    n_total = values.shape[0]
    if n_total == 0 or num_values <= 0:
        return np.array([], dtype=values.dtype)

    k = min(num_values, n_total)
    if random_sample:
        rng = np.random.default_rng(seed)
        idx = rng.choice(n_total, size=k, replace=False)
        return values[idx]
    else:
        return values[:k]

def export_values_from_memory(values: np.ndarray,
                              output_path: str,
                              num_values: int,
                              random_sample: bool = False,
                              seed: Optional[int] = None,
                              apply_abs: bool = False,
                              fmt: str = "{:.10g}") -> np.ndarray:
    """
    Exporta N valores de um array em RAM para um arquivo texto (um por linha).
    Retorna os valores exportados (como np.ndarray) para inspeção no notebook.
    """
    selected = sample_values_in_memory(values, num_values, random_sample, seed)
    if selected.size == 0:
        # cria/zera arquivo
        open(output_path, "w", encoding="utf-8").close()
        print("[INFO] Nenhum valor para exportar.")
        return selected

    if apply_abs:
        selected = np.abs(selected)

    with open(output_path, "w", encoding="utf-8") as f:
        for v in selected:
            f.write(fmt.format(float(v)) + "\n")

    print(f"[OK] Exportados {selected.size} valores para {output_path}")
    return selected

# --------- versão compatível com o seu exportador anterior ---------

def export_values_jsonl_in_ram(input_path: str,
                               output_path: str,
                               num_values: int,
                               random_sample: bool = False,
                               seed: Optional[int] = None,
                               apply_abs: bool = False,
                               fmt: str = "{:.10g}",
                               dtype: str = "float32") -> np.ndarray:
    """
    Carrega todo o JSONL para RAM, extrai todos os escalares,
    e exporta N valores (primeiros ou aleatórios) para um arquivo.
    """
    # 1) carrega linhas
    lines = load_jsonl_lines(input_path)
    # 2) parseia
    objs = parse_jsonl_objects(lines)
    # 3) extrai todos os escalares para um array
    values = extract_all_scalars_from_objects(objs, field="data", dtype=dtype)
    # 4) exporta
    return export_values_from_memory(values, output_path, num_values,
                                     random_sample, seed, apply_abs, fmt)


In [19]:
# %% [markdown]
# # Filtrar pares com probabilidade 1/2500 mantendo alinhamento
# Este notebook lê `wsilu_pairs.txt` e `wsilu_pairs_bin.txt` em paralelo e
# escreve `wsilu_pairs_filtered.txt` e `wsilu_pairs_bin_filtered.txt`,
# copiando cada par de linhas com probabilidade 1/2500.

# %%
import os
import random

# Caminhos de entrada (ajuste se necessário)
pairs_path = "/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs.txt"
bins_path  = "/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs_bin.txt"

# Caminhos de saída
pairs_out_path = "/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs_filtered.txt"
bins_out_path  = "/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs_bin_filtered.txt"

# Parâmetros
denominator = 2500          # probabilidade = 1 / denominator
progress_step = 100_000     # a cada N linhas, mostrar progresso
random_seed = 12345         # defina None para aleatório não reprodutível

# %%
# Preparar RNG
rng = random.Random(random_seed) if random_seed is not None else random

# Sanidade: garantir que diretórios de saída existam
os.makedirs(os.path.dirname(pairs_out_path), exist_ok=True)
os.makedirs(os.path.dirname(bins_out_path), exist_ok=True)

# %%
def filter_paired_files(pairs_in, bins_in, pairs_out, bins_out, denom=2500, step=100_000, rng=random):
    """
    Lê dois arquivos alinhados (mesmo número de linhas, 1-para-1) e grava
    amostras em pares com probabilidade 1/denom, preservando alinhamento.
    """
    total = 0
    kept = 0

    with open(pairs_in, "r", encoding="utf-8") as fp,\
         open(bins_in,  "r", encoding="utf-8") as fb,\
         open(pairs_out, "w", encoding="utf-8") as wp,\
         open(bins_out,  "w", encoding="utf-8") as wb:

        while True:
            lp = fp.readline()
            lb = fb.readline()

            # Checagem de fim de arquivo(s)
            if not lp and not lb:
                break  # ambos acabaram
            if bool(lp) != bool(lb):
                raise RuntimeError(
                    "Desalinhamento detectado: um arquivo terminou antes do outro."
                )

            total += 1

            # amostra com probabilidade 1/denom (mais eficiente que random() < 1/denom)
            if rng.randrange(denom) == 0:
                wp.write(lp)
                wb.write(lb)
                kept += 1

            if total % step == 0:
                print(f"[Progresso] Processadas {total:,} linhas... Mantidas {kept:,}.")

            if(kept >= 100000):
                break

    print(f"\n[Concluído] Total lidas: {total:,} | Total mantidas: {kept:,}")
    return total, kept

# %%
total, kept = filter_paired_files(
    pairs_path, bins_path, pairs_out_path, bins_out_path,
    denom=denominator, step=progress_step, rng=rng
)

print(f"Saídas gravadas em:\n- {pairs_out_path}\n- {bins_out_path}")


[Progresso] Processadas 100,000 linhas... Mantidas 38.
[Progresso] Processadas 200,000 linhas... Mantidas 77.
[Progresso] Processadas 300,000 linhas... Mantidas 119.
[Progresso] Processadas 400,000 linhas... Mantidas 165.
[Progresso] Processadas 500,000 linhas... Mantidas 202.
[Progresso] Processadas 600,000 linhas... Mantidas 236.
[Progresso] Processadas 700,000 linhas... Mantidas 270.
[Progresso] Processadas 800,000 linhas... Mantidas 309.
[Progresso] Processadas 900,000 linhas... Mantidas 341.
[Progresso] Processadas 1,000,000 linhas... Mantidas 374.
[Progresso] Processadas 1,100,000 linhas... Mantidas 412.
[Progresso] Processadas 1,200,000 linhas... Mantidas 441.
[Progresso] Processadas 1,300,000 linhas... Mantidas 474.
[Progresso] Processadas 1,400,000 linhas... Mantidas 512.
[Progresso] Processadas 1,500,000 linhas... Mantidas 559.
[Progresso] Processadas 1,600,000 linhas... Mantidas 588.
[Progresso] Processadas 1,700,000 linhas... Mantidas 619.
[Progresso] Processadas 1,800,000 

In [7]:
min = 1000
max = -1000

with open("/home/ruhan/hwsigmoid_lascas2026/coding_outputs/wsilu_pairs_filtered.txt", "r") as file:
    for l in file:
        input_num = float(l.split()[0])
        # print(input_num)
        if (input_num < min) : min = input_num
        if (input_num > max) : max = input_num

print(min, max)


-7.71484375 5.58203125
